# ✅ Evaluation Notebook: Funding Match Quality

This notebook lets you test multiple user queries or profiles and logs:
- Top semantic matches
- Scores
- GPT recommendations
- Relevance feedback (manual)

Use this for qualitative and semi-quantitative evaluation of your funding finder system.

In [1]:
# 📦 Load dependencies
import os
from dotenv import load_dotenv
from openai import OpenAI
from pinecone import Pinecone
from pprint import pprint

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_ENV = os.getenv("PINECONE_ENV")
assert OPENAI_API_KEY and PINECONE_API_KEY and PINECONE_ENV

client = OpenAI(api_key=OPENAI_API_KEY)
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index("funding-search")

In [2]:
# ✏️ Add test queries here
test_queries = [
    "We are an AI robotics startup in healthcare",
    "Renewable energy solutions provider from Bavaria",
    "EdTech platform using machine learning",
    "Logistics startup using drones and automation",
]

In [3]:
# 🔍 Query Pinecone and GPT for each profile
def run_evaluation(query):
    print(f"\n\n🧠 Query: {query}\n")

    embedding = client.embeddings.create(
        input=[query], model="text-embedding-3-small"
    ).data[0].embedding

    results = index.query(
        vector=embedding,
        top_k=5,
        include_metadata=True,
        namespace="openai-v3"
    )

    chunks = []
    for match in results["matches"]:
        meta = match["metadata"]
        chunks.append(f"{meta.get('name', '')} — {meta.get('description', '')[:300]}")

    # GPT summary
    prompt = f"""
The user profile is:
{query}

Here are the top matched fundings:
{chr(10).join(chunks)}

Suggest the 2–3 most relevant grants and explain why.
"""

    response = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=[
            {"role": "system", "content": "You are a funding advisor."},
            {"role": "user", "content": prompt}
        ]
    )
    print("\n📄 GPT Recommendation:\n")
    print(response.choices[0].message.content)

In [4]:
# ▶️ Run all evaluations
for q in test_queries:
    run_evaluation(q)



🧠 Query: We are an AI robotics startup in healthcare


📄 GPT Recommendation:

For your AI robotics startup in healthcare, selecting the most relevant grants involves focusing on funding opportunities that best align with your sector-specific goals of enhancing AI applications in healthcare settings. The following three grants appear to be the most relevant:

1. **Funding for projects on the topic of "Application of artificial intelligence (AI) in drug discovery"** — This grant directly pertains to the application of AI in a crucial aspect of healthcare—drug discovery. As an AI robotics company in healthcare, your potential innovations or existing technologies could significantly contribute to or benefit from developments in this area. This funding opportunity would support projects that enhance the efficiency and effectiveness of drug discovery processes through advanced AI, which could align well with any platform or tools you are developing that can be applied in this field.

2. **